# 📊 Preprocesado de Datos - Proyectos Horizonte Europa

Este notebook tiene como función la limpieza y preparación de datos administrativos de proyectos europeos del Programa Marco, 9ª Edición (Horizonte Europa).

## Consideraciones Especiales:
- **Datos administrativos**: El dataset no contiene datos experimentales; no se eliminarán los nulos salvo casos muy incompletos.
- **Duplicados**: La variable `Ref.CSIC` se usará como clave primaria. Cualquier duplicado para esta variable será eliminado.
- **Variables string**: Muchas columnas numéricas son en realidad etiquetas (ej: `Ref.UE`, `Centro`). NO deben usarse para análisis estadisticos multivariable.
- **Normalización**: `Nombre centro IP` contiene etiquetas distintas para el mismo centro; debe normalizarse usando `Centro` como referencia.

## Contenido:
1. Carga de Datos
2. Renombrar variables ambiguas
3. Clasificación de Variables
4. Eliminación de Duplicados
5. Normalización de Nombres de Centros
6. Conversión de Variables temporales
7. Limpieza de Strings
8. Variables Derivadas: Transformar Areas Científicas antiguas a nuevas
9. Variables Derivadas: Otras variables derivadas de interés
10. Resumen Final del Preprocesado
11. Exportación de Datos Limpios

In [17]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Carga de Datos

In [18]:
#carga de datos desde hoja excel
df = pd.read_excel(
    '../data/9PM_bootcamp1.xlsx',
    dtype={'Centro': str, 'Ref.UE': str}
)

print(f"📁 Datos cargados: {df.shape[0]} filas (proyectos) y {df.shape[1]} columnas")
print(f"\n📋 Columnas encontradas:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")



📁 Datos cargados: 719 filas (proyectos) y 20 columnas

📋 Columnas encontradas:
   1. Ref.CSIC
   2. Ref.UE
   3. Situación
   4. Programa
   5. Acción clave
   6. Título
   7. Comienzo
   8. Final
   9. Duración (meses)
  10. Concedido
  11. CSIC
  12. España (no CSIC)
  13. Total (Csic, Esp. y otros)
  14. Cód.área
  15. Acrónimo del proyecto
  16. Convocatoria
  17. Centro
  18. Nombre Centro IP
  19. Resumen
  20. Keywords


# 2. Renombrar variables ambiguas

Se renombrarán las siguientes variables, actualmente con nombres ambiguos, para mayor claridad:

| **Variable Original** | **Nuevo Nombre** |
|----------------------|------------------|
| `Concedido` | `Importe Concedido` |
| `CSIC` | `Participantes CSIC` |
| `España (no CSIC)` | `Participantes España (no CSIC)` |
| `Total (Csic, Esp. y otros)` | `Total Participantes` |





In [19]:

renombres = {
    'Concedido': 'Importe Concedido',
    'CSIC': 'Participantes CSIC',
    'España (no CSIC)': 'Participantes España (no CSIC)',
    'Total (Csic, Esp. y otros)': 'Total Participantes'
}

# Aplicar renombres
df = df.rename(columns=renombres)

print ("Nombres de variables actualizadas:")
# Mostrar confirmación de cambios
for original, nuevo in renombres.items():
    if original in df.columns or nuevo in df.columns:
        print(f"   ✅ '{original}' → '{nuevo}'")


# Verificar nuevos nombres
print("\n📋 VARIABLES RENOMBRADAS DISPONIBLES:")
for nuevo_nombre in renombres.values():
    if nuevo_nombre in df.columns:
        print(f"   ✓ {nuevo_nombre}")

Nombres de variables actualizadas:
   ✅ 'Concedido' → 'Importe Concedido'
   ✅ 'CSIC' → 'Participantes CSIC'
   ✅ 'España (no CSIC)' → 'Participantes España (no CSIC)'
   ✅ 'Total (Csic, Esp. y otros)' → 'Total Participantes'

📋 VARIABLES RENOMBRADAS DISPONIBLES:
   ✓ Importe Concedido
   ✓ Participantes CSIC
   ✓ Participantes España (no CSIC)
   ✓ Total Participantes


## 3. Clasificación de Variables

El dataset contiene variables de distinto tipo: identificadores, variables categóricas, descriptivas... y solo algunas numéricas. Es necesario clasificarlas correctamente y tener en cuenta esta clasificación para su análisis.


In [20]:
# Definir clasificación de variables según su función REAL en datos administrativos
variables_config = {
    'identificadores': ['Ref.CSIC', 'Ref.UE', 'Acrónimo del proyecto'],  # Códigos únicos
    'categoricas': ['Situación', 'Programa', 'Acción clave', 'Convocatoria', 'Cód.área','Centro', 'Nombre Centro IP'],  # Variables categóricas
    'descriptivas': ['Título', 'Resumen', 'Keywords'],  # Texto libre para búsqueda
    'numericas': ['Importe Concedido','Duración (meses)',  'Participantes CSIC', 'Participantes España (no CSIC)', 'Total Participantes'], 
    'temporales': ['Comienzo', 'Final'],  # Fechas
}

print("📊 CLASIFICACIÓN DE VARIABLES SEGÚN TIPO:")
print("=" * 80)
for tipo, cols in variables_config.items():
    print(f"\n🔹 {tipo.upper()}:")
    for col in cols:
        if col in df.columns:
            print(f"   ✓ {col}")
        else:
            print(f"   ✗ {col} (no encontrada)")

# Verificar si hay columnas no clasificadas
columnas_no_clasificadas = set(df.columns) - {col for cols in variables_config.values() for col in cols}
if columnas_no_clasificadas:
    print(f"\n⚠️ COLUMNAS NO CLASIFICADAS:")
    for col in columnas_no_clasificadas:
        print(f"   ? {col}")

📊 CLASIFICACIÓN DE VARIABLES SEGÚN TIPO:

🔹 IDENTIFICADORES:
   ✓ Ref.CSIC
   ✓ Ref.UE
   ✓ Acrónimo del proyecto

🔹 CATEGORICAS:
   ✓ Situación
   ✓ Programa
   ✓ Acción clave
   ✓ Convocatoria
   ✓ Cód.área
   ✓ Centro
   ✓ Nombre Centro IP

🔹 DESCRIPTIVAS:
   ✓ Título
   ✓ Resumen
   ✓ Keywords

🔹 NUMERICAS:
   ✓ Importe Concedido
   ✓ Duración (meses)
   ✓ Participantes CSIC
   ✓ Participantes España (no CSIC)
   ✓ Total Participantes

🔹 TEMPORALES:
   ✓ Comienzo
   ✓ Final


Veamos el Data type de cada variable:

In [21]:
# Información detallada del dataset
print("📋 INFORMACIÓN DEL DATASET:")
print("=" * 70)
df.info()

📋 INFORMACIÓN DEL DATASET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Ref.CSIC                        695 non-null    object 
 1   Ref.UE                          694 non-null    object 
 2   Situación                       695 non-null    object 
 3   Programa                        692 non-null    object 
 4   Acción clave                    692 non-null    object 
 5   Título                          695 non-null    object 
 6   Comienzo                        691 non-null    object 
 7   Final                           691 non-null    object 
 8   Duración (meses)                690 non-null    float64
 9   Importe Concedido               695 non-null    float64
 10  Participantes CSIC              693 non-null    float64
 11  Participantes España (no CSIC)  693 non-null    float64
 12  Total Par


Para evitar errores, es preferible modificar el tipo de dato para las variables numéricas.

| **Variable** | **Tipo Actual** | **Tipo Objetivo** |
|-------------|----------------|-------------------|
| `Duración (meses)` | `float64` | `int64` |
| `Importe Concedido` | `float64` | `float64` (Se mantiene igual)|
| `Participantes CSIC` | `float64` | `int64` |
| `Participantes España (no CSIC)` | `float64` | `int64` |
| `Total Participantes` | `float64` | `int64` |
| `Cód.área` | `float64` | `object` Código identificador, no es variable numérica|



> **⚠️ Nota importante**: Aunque `Centro` y `Cód.área` contengan números, representan códigos identificadores y deben tratarse como texto para evitar operaciones matemáticas incorrectas.


In [22]:

# 1. Variables enteras (sin decimales)
variables_enteras = ['Duración (meses)', 'Participantes CSIC', 'Participantes España (no CSIC)', 'Total Participantes']

for var in variables_enteras:
    if var in df.columns:
        df[var] = df[var].astype('Int64')  # Int64 permite valores nulos
        print(f"   ✅ {var} → int64")

# 2. Variables que son códigos identificadores (Cód.área)

if 'Cód.área' in df.columns:
    df['Cód.área'] = pd.to_numeric(df['Cód.área'], errors='coerce').astype('Int64')
    df['Cód.área'] = df['Cód.área'].astype(str).replace('<NA>', pd.NA)
    print(f"   ✅ Cód.área → int64 → object (código identificador)")

# 3. Importe Concedido se mantiene como float64
print(f"   ✅ Importe Concedido → float64 (se mantiene)")

print("\n📊 Verificación de tipos de datos:")
tipos_relevantes = ['Duración (meses)', 'Importe Concedido', 'Participantes CSIC', 
                   'Participantes España (no CSIC)', 'Total Participantes', 'Centro', 'Cód.área']

for var in tipos_relevantes:
    if var in df.columns:
        print(f"   {var}: {df[var].dtype}")


   ✅ Duración (meses) → int64
   ✅ Participantes CSIC → int64
   ✅ Participantes España (no CSIC) → int64
   ✅ Total Participantes → int64
   ✅ Cód.área → int64 → object (código identificador)
   ✅ Importe Concedido → float64 (se mantiene)

📊 Verificación de tipos de datos:
   Duración (meses): Int64
   Importe Concedido: float64
   Participantes CSIC: Int64
   Participantes España (no CSIC): Int64
   Total Participantes: Int64
   Centro: object
   Cód.área: object


Ahora veamos si los cambios se han aplicado correctamente:

In [23]:
# Vista previa de datos
print("🔍 PRIMERAS FILAS DEL DATASET:")
print("=" * 70)
display(df.head())

print("\n📊 INFORMACIÓN BÁSICA:")
print(f"Total proyectos: {len(df)}")

🔍 PRIMERAS FILAS DEL DATASET:


,Ref.CSIC,Ref.UE,Situación,Programa,Acción clave,Título,Comienzo,Final,Duración (meses),Importe Concedido,Participantes CSIC,Participantes España (no CSIC),Total Participantes,Cód.área,Acrónimo del proyecto,Convocatoria,Centro,Nombre Centro IP,Resumen,Keywords
0,EUROHPC/1238,101196247,VIGENTE,JOINT UNDERTAKING,JOINT UNDERTAKING,Europe-India Partnership for Scientific High-P...,01/02/2025,31/01/2028,36,302687.50,1,2,10,8903,GANANA,HORIZON-EUROHPC-JU-2023-INCO-06-01,030102,GEOCIENCIAS BARCELONA,HIGH-PERFORMANCE COMPUTING (HPC) IS A MAJOR EN...,NaN
1,HE/CBE/0653,101214822,VIGENTE,JOINT UNDERTAKING,JOINT UNDERTAKING,Bio-based in Soil applications with Optimal bi...,01/09/2025,31/08/2029,48,273837.50,1,6,16,8908,SOUL,HORIZON-JU-CBE-2024,020401,INSTO. CATALISIS Y PETROLEOQUIMICA,THE USE OF PLASTIC PRODUCTS HAS INCREASED SIGN...,IMPROVED CONTROLLED BIODEGRADABILIY; IMPROVED ...
2,HE/CBE/0653,101214822,VIGENTE,JOINT UNDERTAKING,JOINT UNDERTAKING,Bio-based in Soil applications with Optimal bi...,01/09/2025,31/08/2029,48,273837.50,1,6,16,8908,SOUL,HORIZON-JU-CBE-2024,020401,INSTO. CATALISIS Y PETROLEOQUIMICA,THE USE OF PLASTIC PRODUCTS HAS INCREASED SIGN...,IMPROVED CONTROLLED BIODEGRADABILIY; IMPROVED ...
3,HE/CLEANH2/0431,101137792,VIGENTE,JTI-CLEANH2,JTI-CLEANH2,A novel multi-stage steam gasification and syn...,01/01/2024,31/12/2027,48,220047.50,1,10,16,8908,HYIELD,HORIZON-JTI-CLEANH2-2023-1,020403,INSTO. CARBOQUIMICA,EUROPE FACES THE JOINT CHALLENGE OF DECARBONIS...,NaN
4,HE/CLEANH2/0571,101137756,VIGENTE,JTI-CLEANH2,JTI-CLEANH2,Carbon-negative pressurized hydrogen productio...,01/10/2024,30/09/2028,48,224156.25,0,0,0,8908,CARMA-H2,HORIZON-JTI-CLEANH2-2023-1,020166,INSTO. TECNOLOGIA QUIMICA,CARMA-H2 WILL ENABLE HIGHLY ATTRACTIVE HYDROGE...,CARBON NEGATIVE ELECTROCHEMICAL HYDROGEN PRODU...



📊 INFORMACIÓN BÁSICA:
Total proyectos: 719


Ahora las variables numéricas tienen el formato correcto

## 4. Eliminación de Duplicados

Al reflejar cada registro un proyecto individual, los campos que actuan como clave (`Ref. CSIC`, `Ref. UE`, `Acrónimo del proyecto`) no deberían repetirse. A continuación se detectan y eliminan posibles duplicados en base a `Ref.CSIC` (anteriormente se eliminan los nulos).

In [24]:
# Eliminar filas con Ref.CSIC nulo
print("🧹 ELIMINACIÓN DE NULOS Y DUPLICADOS EN Ref.CSIC")
print("=" * 60)

# 1. Eliminar filas donde Ref.CSIC es nulo
filas_antes = len(df)
df = df.dropna(subset=['Ref.CSIC'])
filas_despues_nulos = len(df)
nulos_eliminados = filas_antes - filas_despues_nulos

print(f"✅ Filas con Ref.CSIC nulo eliminadas: {nulos_eliminados}")

# 2. Eliminar duplicados por Ref.CSIC (mantener primera ocurrencia)
df = df.drop_duplicates(subset=['Ref.CSIC'], keep='first')
filas_final = len(df)
duplicados_eliminados = filas_despues_nulos - filas_final

print(f"✅ Duplicados por Ref.CSIC eliminados: {duplicados_eliminados}")
print(f"📊 Dataset final: {filas_final} proyectos únicos")



🧹 ELIMINACIÓN DE NULOS Y DUPLICADOS EN Ref.CSIC
✅ Filas con Ref.CSIC nulo eliminadas: 24
✅ Duplicados por Ref.CSIC eliminados: 1
📊 Dataset final: 694 proyectos únicos


## 5. Normalización de Nombres de Centros

**CRÍTICO**: `nombre centro IP` puede tener variaciones (abreviaturas).  
Normalizar usando `Centro` (código) como referencia.

In [25]:
# Explorar relación Centro - nombre centro IP
if 'Centro' in df.columns and 'Nombre Centro IP' in df.columns:
    print("🔍 ANÁLISIS DE NOMBRES DE CENTROS:")
    print("=" * 70)
    
    # Agrupar por código de centro y ver variaciones de nombres
    centro_nombres = df.groupby('Centro')['Nombre Centro IP'].unique()
    
    # Detectar centros con múltiples nombres
    centros_multiples = {centro: nombres for centro, nombres in centro_nombres.items() 
                         if len(nombres) > 1}
    
    if centros_multiples:
        print(f"\n⚠️ Se encontraron {len(centros_multiples)} centros con múltiples variaciones de nombre:")
        for centro, nombres in list(centros_multiples.items())[:10]:  # Mostrar primeros 10
            print(f"\n  Centro código: {centro}")
            for nombre in nombres:
                print(f"    - {nombre}")
    else:
        print("✅ No se encontraron variaciones en nombres de centros")

🔍 ANÁLISIS DE NOMBRES DE CENTROS:

✅ No se encontraron variaciones en nombres de centros
✅ No se encontraron variaciones en nombres de centros


In [26]:
# Normalizar nombres de centros usando el nombre más frecuente por código
if 'Centro' in df.columns and 'Nombre Centro IP' in df.columns:
    # Crear mapeo: código centro → nombre más frecuente
    centro_nombre_map = {}
    
    for centro in df['Centro'].unique():
        if pd.notna(centro):
            # Obtener nombre más frecuente para este centro
            nombres = df[df['Centro'] == centro]['Nombre Centro IP']
            nombre_frecuente = nombres.mode()[0] if len(nombres.mode()) > 0 else nombres.iloc[0]
            centro_nombre_map[centro] = nombre_frecuente
    
    # Aplicar normalización
    df['Nombre Centro IP Normalizado'] = df['Centro'].map(centro_nombre_map)
    
    # Contar cambios
    cambios = (df['Nombre Centro IP'] != df['Nombre Centro IP Normalizado']).sum()
    print(f"\n✅ Normalización completada:")
    print(f"   Total de registros con nombre normalizado: {cambios}")
    print(f"   Centros únicos: {df['Nombre Centro IP Normalizado'].nunique()}")

    # Mostrar ejemplo de normalización
    if cambios > 0:
        print("\n📊 Ejemplos de normalización:")
        ejemplos = df[df['Nombre Centro IP'] != df['Nombre Centro IP Normalizado']][
            ['Centro', 'Nombre Centro IP', 'Nombre Centro IP Normalizado']
        ].head(5)
        display(ejemplos)


✅ Normalización completada:
   Total de registros con nombre normalizado: 0
   Centros únicos: 118


## 6. Conversión de Variables temporales

Modificar el tipo de dato de las variables temporales para asegurarse de que tengan el tipo correcto

In [27]:
# Convertir variables temporales a formato datetime
print("🗓️ CONVERSIÓN DE VARIABLES TEMPORALES:")
print("=" * 60)

# Variables temporales a convertir
variables_temporales = ['Comienzo', 'Final']

for var in variables_temporales:
    if var in df.columns:
        # Convertir a datetime
        df[var] = pd.to_datetime(df[var], errors='coerce')
        print(f"   ✅ {var} → datetime64[ns]")

# Verificar conversión
print("\n📊 Verificación de tipos de datos temporales:")
for var in variables_temporales:
    if var in df.columns:
        print(f"   {var}: {df[var].dtype}")
        # Mostrar rango de fechas
        fecha_min = df[var].min()
        fecha_max = df[var].max()
        print(f"     Rango: {fecha_min} a {fecha_max}")

print("\n✅ Conversión de variables temporales completada")

🗓️ CONVERSIÓN DE VARIABLES TEMPORALES:
   ✅ Comienzo → datetime64[ns]
   ✅ Final → datetime64[ns]

📊 Verificación de tipos de datos temporales:
   Comienzo: datetime64[ns]
     Rango: 2021-01-01 00:00:00 a 2026-01-09 00:00:00
   Final: datetime64[ns]
     Rango: 2022-12-31 00:00:00 a 2034-05-31 00:00:00

✅ Conversión de variables temporales completada


## 7. Limpieza de Strings

Eliminar espacios y normalizar formato

In [28]:
# Limpiar columnas de texto
text_columns = df.select_dtypes(include=['object']).columns

for col in text_columns:
    if col in df.columns and df[col].dtype == 'object':
        # Eliminar espacios al inicio y final
        df[col] = df[col].str.strip()
        # Reemplazar múltiples espacios por uno solo
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

print(f"✅ Limpieza de {len(text_columns)} columnas de texto completada")

✅ Limpieza de 14 columnas de texto completada


## 8. Variables Derivadas: Transformar Areas Científicas antiguas a nuevas

La variable `Cód.Área` hace referencia a las antiguas áreas científicas del CSIC. Se transformarán a las nuevas categorías: Vida, Materia y Sociedad (incluyendo tambien las categorías Central y Desconocido).

In [29]:
# Cargar el diccionario de áreas desde el archivo maestros
areas_dict = pd.read_excel('../data/maestros.xlsx', sheet_name='Hoja1')

print("🔍 DICCIONARIO DE ÁREAS CIENTÍFICAS:")
print("=" * 50)
display(areas_dict.head())

# Convertir Cód.área a string para hacer el mapeo (ambas columnas deben ser del mismo tipo)
areas_dict['Cód.área'] = areas_dict['Cód.área'].astype(str)

# Crear el mapeo de código antiguo a nueva categoría
codigo_a_categoria = dict(zip(areas_dict['Cód.área'], areas_dict['Area']))

print(f"\n📊 MAPEO DE TRANSFORMACIÓN:")
print("Cód.área → Area")
for codigo, categoria in list(codigo_a_categoria.items())[:10]:  # Mostrar primeros 10
    print(f"  {codigo} → {categoria}")

# Transformar la variable Cód.área a las nuevas categorías
# IMPORTANTE: La nueva variable se llama 'Area' (sin tilde)
df['Area'] = df['Cód.área'].map(codigo_a_categoria)

# Verificar la transformación
print(f"\n✅ TRANSFORMACIÓN COMPLETADA:")
print(f"   Total códigos únicos originales: {df['Cód.área'].nunique()}")
print(f"   Total categorías nuevas: {df['Area'].nunique()}")

# Mostrar distribución de nuevas categorías
print(f"\n📈 DISTRIBUCIÓN POR NUEVA CATEGORÍA:")
distribucion = df['Area'].value_counts()
for categoria, count in distribucion.items():
    print(f"   {categoria}: {count} proyectos")

# Verificar si hay códigos sin mapear
sin_mapear = df[df['Area'].isna()]['Cód.área'].unique()
if len(sin_mapear) > 0:
    print(f"\n⚠️ CÓDIGOS SIN MAPEAR: {sin_mapear}")
    print(f"   Total registros sin mapear: {df['Area'].isna().sum()}")

print(f"\n🔍 VISTA PREVIA DEL DATASET CON NUEVA VARIABLE:")
print("=" * 70)
display(df[['Ref.CSIC', 'Título', 'Cód.área', 'Area', 'Centro']].head())

🔍 DICCIONARIO DE ÁREAS CIENTÍFICAS:


,Cód.área,Area
0,8901,SOCIEDAD
1,8902,VIDA
2,8903,VIDA
3,8904,VIDA
4,8905,MATERIA



📊 MAPEO DE TRANSFORMACIÓN:
Cód.área → Area
  8901 → SOCIEDAD
  8902 → VIDA
  8903 → VIDA
  8904 → VIDA
  8905 → MATERIA
  8906 → MATERIA
  8907 → VIDA
  8908 → MATERIA
  8909 → CENTRAL
  8900 → DESCONOCIDO

✅ TRANSFORMACIÓN COMPLETADA:
   Total códigos únicos originales: 10
   Total categorías nuevas: 5

📈 DISTRIBUCIÓN POR NUEVA CATEGORÍA:
   VIDA: 314 proyectos
   MATERIA: 244 proyectos
   DESCONOCIDO: 98 proyectos
   SOCIEDAD: 36 proyectos
   CENTRAL: 2 proyectos

🔍 VISTA PREVIA DEL DATASET CON NUEVA VARIABLE:


,Ref.CSIC,Título,Cód.área,Area,Centro
0,EUROHPC/1238,Europe-India Partnership for Scientific High-P...,8903,VIDA,030102
1,HE/CBE/0653,Bio-based in Soil applications with Optimal bi...,8908,MATERIA,020401
3,HE/CLEANH2/0431,A novel multi-stage steam gasification and syn...,8908,MATERIA,020403
4,HE/CLEANH2/0571,Carbon-negative pressurized hydrogen productio...,8908,MATERIA,020166
5,HE/CLEANH2/0576,DIAGNOSTIC TOOLS AND RISK PROTOCOLS TO ACCELER...,8903,VIDA,030502


## 9. Variables Derivadas: Otras variables derivadas de interés.

Con la intención de comparar y analizar mejor los datos, se crearán también otras variables derivadas a partir de las existentes:

| **Variable Derivada** | **Tipo** | **Descripción** |
|----------------------|----------|-----------------|
| **Año Inicio** | `object` | Año de inicio del proyecto |
| **Año Fin** | `object` | Año de finalización del proyecto |
| **Presupuesto Mensual** | `float64` | Presupuesto total dividido entre la duración en meses |
| **Duración (categoría)** | `object` | Clasificación de proyectos por duración (Corto/Medio/Largo) |
| **Presupuesto (categoría)** | `object` | Clasificación de proyectos por importe concedido |

Estas variables facilitarán análisis temporales, comparaciones por períodos y segmentación de proyectos según magnitud y duración.


In [30]:
# Crear variables derivadas
print("🔧 CREACIÓN DE VARIABLES DERIVADAS:")
print("=" * 50)

# 1. Año Inicio y Año Fin (convertir a int primero para eliminar decimales, luego a string)
df['Año Inicio'] = df['Comienzo'].dt.year.astype('Int64').astype(str)
df['Año Fin'] = df['Final'].dt.year.astype('Int64').astype(str)
print("   ✅ Año Inicio y Año Fin creadas")

# 2. Presupuesto Mensual
df['Presupuesto Mensual'] = df['Importe Concedido'] / df['Duración (meses)']
df['Presupuesto Mensual'] = df['Presupuesto Mensual'].round(2)
print("   ✅ Presupuesto Mensual creada")

# 3. Duración (categoría)
df['Duración (categoría)'] = pd.cut(df['Duración (meses)'], 
                                   bins=[0, 12, 36, float('inf')], 
                                   labels=['Corto', 'Medio', 'Largo'])
print("   ✅ Duración (categoría) creada")

# 4. Presupuesto (categoría)
df['Presupuesto (categoría)'] = pd.cut(df['Importe Concedido'],
                                      bins=[0, 150000, 500000, float('inf')],
                                      labels=['Pequeño', 'Mediano', 'Grande'])
print("   ✅ Presupuesto (categoría) creada")

print(f"\n📊 Variables derivadas creadas: 5")
print(f"📊 Total de variables en el dataset: {df.shape[1]}")

# Vista previa de las nuevas variables
print(f"\n🔍 VISTA PREVIA DE VARIABLES DERIVADAS:")
display(df[['Ref.CSIC', 'Título', 'Año Inicio', 'Año Fin', 'Presupuesto Mensual', 
           'Duración (categoría)', 'Presupuesto (categoría)']].head())

🔧 CREACIÓN DE VARIABLES DERIVADAS:
   ✅ Año Inicio y Año Fin creadas
   ✅ Presupuesto Mensual creada
   ✅ Duración (categoría) creada
   ✅ Presupuesto (categoría) creada

📊 Variables derivadas creadas: 5
📊 Total de variables en el dataset: 27

🔍 VISTA PREVIA DE VARIABLES DERIVADAS:


,Ref.CSIC,Título,Año Inicio,Año Fin,Presupuesto Mensual,Duración (categoría),Presupuesto (categoría)
0,EUROHPC/1238,Europe-India Partnership for Scientific High-P...,2025,2028,8407.99,Medio,Mediano
1,HE/CBE/0653,Bio-based in Soil applications with Optimal bi...,2025,2029,5704.95,Largo,Mediano
3,HE/CLEANH2/0431,A novel multi-stage steam gasification and syn...,2024,2027,4584.32,Largo,Mediano
4,HE/CLEANH2/0571,Carbon-negative pressurized hydrogen productio...,2024,2028,4669.92,Largo,Mediano
5,HE/CLEANH2/0576,DIAGNOSTIC TOOLS AND RISK PROTOCOLS TO ACCELER...,2025,2028,9483.55,Largo,Mediano


## 10. Resumen Final del Preprocesado

In [33]:
# Resumen final del preprocesado de datos
print("📊 RESUMEN FINAL DEL PREPROCESADO DE DATOS")
print("=" * 60)

# Información básica del dataset
print(f"🔢 INFORMACIÓN GENERAL:")
print(f"   Total de proyectos: {len(df):,}")
print(f"   Total de variables: {df.shape[1]}")
print(f"   Duplicados eliminados: {duplicados_eliminados}")
print(f"   Nulos en Ref.CSIC eliminados: {nulos_eliminados}")

# Rango temporal (Año Inicio y Año Fin ya son strings, convertir a int para cálculos)
años_inicio = df['Año Inicio'].dropna().replace('<NA>', pd.NA).dropna().astype(int)
años_fin = df['Año Fin'].dropna().replace('<NA>', pd.NA).dropna().astype(int)
print(f"\n📅 RANGO TEMPORAL:")
print(f"   Año inicio: {años_inicio.min()} - {años_inicio.max()}")
print(f"   Año finalización: {años_fin.min()} - {años_fin.max()}")

# Información presupuestaria
presupuesto_total = df['Importe Concedido'].sum()
presupuesto_medio = df['Importe Concedido'].mean()
print(f"\n💰 INFORMACIÓN PRESUPUESTARIA:")
print(f"   Presupuesto total: {presupuesto_total:,.2f} €")
print(f"   Presupuesto medio por proyecto: {presupuesto_medio:,.2f} €")

# Distribución por área científica
print(f"\n🔬 DISTRIBUCIÓN POR ÁREA CIENTÍFICA:")
for area, count in df['Area'].value_counts().items():
    porcentaje = (count / len(df)) * 100
    print(f"   {area}: {count} proyectos ({porcentaje:.1f}%)")

# Variables creadas/transformadas
print(f"\n🔧 TRANSFORMACIONES REALIZADAS:")
print(f"   ✅ Variables renombradas: {len(renombres)}")
print(f"   ✅ Nombres de centros normalizados: {cambios} registros")
print(f"   ✅ Variables derivadas creadas: 5")
print(f"   ✅ Conversión de tipos de datos: completada")
print(f"   ✅ Limpieza de texto: {len(text_columns)} columnas")

print(f"\n✅ Dataset preprocesado listo para análisis")

📊 RESUMEN FINAL DEL PREPROCESADO DE DATOS
🔢 INFORMACIÓN GENERAL:
   Total de proyectos: 694
   Total de variables: 27
   Duplicados eliminados: 1
   Nulos en Ref.CSIC eliminados: 24

📅 RANGO TEMPORAL:
   Año inicio: 2021 - 2026
   Año finalización: 2022 - 2034

💰 INFORMACIÓN PRESUPUESTARIA:
   Presupuesto total: 336,424,615.07 €
   Presupuesto medio por proyecto: 484,761.69 €

🔬 DISTRIBUCIÓN POR ÁREA CIENTÍFICA:
   VIDA: 314 proyectos (45.2%)
   MATERIA: 244 proyectos (35.2%)
   DESCONOCIDO: 98 proyectos (14.1%)
   SOCIEDAD: 36 proyectos (5.2%)
   CENTRAL: 2 proyectos (0.3%)

🔧 TRANSFORMACIONES REALIZADAS:
   ✅ Variables renombradas: 4
   ✅ Nombres de centros normalizados: 0 registros
   ✅ Variables derivadas creadas: 5
   ✅ Conversión de tipos de datos: completada
   ✅ Limpieza de texto: 14 columnas

✅ Dataset preprocesado listo para análisis


## 11. Exportación de Datos Limpios

In [34]:
# Guardar dataset preprocesado
df.to_csv('../data/9PM_bootcamp_clean.csv', index=False, encoding='utf-8')
df.to_excel('../data/9PM_bootcamp_clean.xlsx', index=False)

print("✅ Datos exportados exitosamente:")
print("   📁 ../data/9PM_bootcamp_clean.csv")
print("   📁 ../data/9PM_bootcamp_clean.xlsx")
print(f"\n📊 Total de proyectos exportados: {len(df):,}")

✅ Datos exportados exitosamente:
   📁 ../data/9PM_bootcamp_clean.csv
   📁 ../data/9PM_bootcamp_clean.xlsx

📊 Total de proyectos exportados: 694
